In [1]:
# %pip install llama-index
# %pip install transformers accelerate bitsandbytes
# %pip install llama-index-readers-web
# %pip install llama-index-llms-huggingface
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-program-openai
# %pip install llama-index-agent-openai

Setup

Loading Stage

In [2]:
from llama_index.core.response.notebook_utils import display_response
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
#pdf files are stored in the directory 'data'
documents = SimpleDirectoryReader("data").load_data()

Indexing Stage

LLM

In [2]:
# %pip install transformers accelerate

In [4]:
# huggingface api token for downloading llama2
hf_token = "add_token_here"

In [5]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate(" [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Embedding Model

In [6]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Storing Stage

In [7]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)

Query Stage

In [8]:
query_engine = vector_index.as_query_engine()

response = query_engine.query("What is data responsibility in humanitarian action")

display_response(response)

**`Final Response:`** Based on the context information provided, data responsibility in humanitarian action refers to the safe, ethical, and effective management of personal and non-personal data for operational response. It is critical to fulfilling the commitments related to Accountability to Affected People (AAP) in humanitarian action. Data responsibility includes ensuring that data management activities do not harm affected people and that appropriate mechanisms are in place for collecting, analyzing, referring, and actioning complaints and feedback. Common data management activities related to AAP include communicating with communities through timely and relevant information, mechanisms for collecting, analyzing, referring, and actioning complaints and feedback, systems for tracking community perceptions, rumors, and misinformation, disinformation and hate speech, information management activities to inform the delivery and coordination of assistance, and information ecosystem analysis to capture the relationship between information supply and information demand to and from communities.

In [9]:
query_engine = vector_index.as_query_engine(similarity_top_k=5)
response = query_engine.query("What is data responsibility in humanitarian action")

display_response(response)

**`Final Response:`** Based on the context information provided, data responsibility in humanitarian action refers to the safe, ethical, and effective management of data for operational response. It encompasses both data protection (safeguarding the right to privacy) and data security (measures to preserve the confidentiality, availability, and integrity of personal and non-personal data). Data responsibility is critical in the humanitarian sector, as a lack of robust data security can undermine the ability to deliver assistance and protection to affected people, maintain duty of care for staff, and operate effectively in complex environments.

The context information provides guidance notes and tip sheets on specific issues related to data responsibility in practice, such as data ethics, statistical disclosure control, and cybersecurity. It also highlights the importance of adopting relevant policies, guidelines, and risk-related mitigation procedures, investing in responsible data management, and promoting actions for data responsibility among staff and partners.

Overall, data responsibility in humanitarian action is essential for ensuring that data is managed in a way that promotes trust, accountability, and effective response to humanitarian crises.

In [10]:
query_engine = vector_index.as_query_engine()

response = query_engine.query("What is the systematic bias in typoon impact model")

display_response(response)

**`Final Response:`** Based on the context information provided, the systematic bias in the Typhoon Impact Model is "Lack of Transparency." The model is open-source and has undergone technical review, but it is missing some documentation, which can lead to a lack of transparency in the modeling process, rationale for algorithm selection, and fitness for purpose. This can result in a bias in the model's predictions and decisions, potentially leading to unfair allocation of funding and activities.

In [11]:
query_engine = vector_index.as_query_engine()

response = query_engine.query("What is guidance note say on poorly secured network")

display_response(response)

**`Final Response:`** Based on the provided context information, the guidance note suggests the following best practices for securing a network:

1. Keep software and operating systems up to date: The guidance note advises checking regularly for updates and enabling automatic updates to ensure the latest security patches are applied.
2. Use web browsers that receive automatic security updates: The note recommends using web browsers that receive automatic security updates to minimize the risk of vulnerabilities.
3. Avoid using software for which future maintenance is likely not to be available: The guidance note advises avoiding software that is no longer receiving updates, as it may be more vulnerable to security risks.
4. Shut down devices regularly or when prompted by the system: The note suggests shutting down devices regularly or when prompted by the system to enable updating and protect against attacks.
5. Be cautious when clicking on links or attachments: The guidance note advises being cautious when clicking on links or attachments, even from trusted senders, and always checking the URL and sender information before opening.
6. Use mobile devices responsibly: The note suggests using separate devices for work purposes and keeping work devices in a secure place at all times.

In [12]:
query_engine = vector_index.as_query_engine()

response = query_engine.query("What is Austen's Critique of Patriarchal Societies?")

display_response(response)

**`Final Response:`** Based on the provided context information, Jane Austen's critique of patriarchal societies can be inferred as follows:

1. Limited marriage prospects for women: Austen highlights the dependence of women on male relatives for financial security and social status. In "Pride and Prejudice," the Bennet sisters face pressure to marry well due to their limited inheritance rights.
2. Gender roles and expectations: Austen's novels examine the social and economic constraints on women's choices and marriage prospects, underscoring the limitations placed on women by patriarchal societies.
3. Dependence on male relatives: Austen's works emphasize the dependence of women on male relatives for financial security and social status, highlighting the economic vulnerabilities of women in patriarchal societies.
4. Lack of agency and autonomy: Austen's novels underscore the limited agency and autonomy of women in patriarchal societies, where women were often seen as subservient to men.

In summary, Austen's critique of patriarchal societies through her works highlights the economic, social, and cultural limitations placed on women, emphas

Change in Response Mode: In the "refine" response mode, the model iteratively refines the initial response based on additional context.

In [13]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("What is Austen's Critique of Patriarchal Societies?")

display_response(response)

**`Final Response:`** Based on the provided context, we can refine the original answer as follows:

Jane Austen's critique of patriarchal societies in her novels "Pride and Prejudice" and "Sense and Sensibility" can be summarized as follows:

1. Limited choices and marriage prospects for women: Austen highlights the economic and social constraints placed on women during the Regency era in England, including the limited options available to them in terms of marriage and financial stability. The novels emphasize the dependence of women on their male relatives for financial security and social status.
2. Dependence on male relatives for financial stability: Austen critiques the societal expectations placed on women to marry well due to the limited inheritance rights of women. The novels showcase the dependence of women on their male relatives for financial stability, as seen in the character of Charlotte Lucas, who marries Mr. Collins for financial stability.
3. Emergence of feminist literature: Austen's works mark the emergence of feminist literature, with "Pride and Prejudice" and "Sense and Sensibility" being among the first novels to critically examine the gender roles and

Change in Response Mode: The "compact" response mode focuses on providing a concise and to-the-point answer.

In [14]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("What is Austen's Critique of Patriarchal Societies?")

display_response(response)

**`Final Response:`** Based on the context information provided, Jane Austen's critique of patriarchal societies can be inferred as follows:

1. Limited marriage prospects: Austen highlights the dependence of women on male relatives for financial security and social status. In "Pride and Prejudice," the Bennet sisters face pressure to marry well due to their limited inheritance rights.
2. Gender roles and expectations: Austen critiques the rigid gender roles and expectations placed on women in patriarchal societies. In "Sense and Sensibility," the Dashwood sisters must navigate societal expectations and financial dependence on male relatives after their father's death.
3. Economic vulnerabilities: Austen examines the economic vulnerabilities of female characters in patriarchal societies. In "Pride and Prejudice," Charlotte Lucas marries Mr. Collins for financial stability, illustrating the limited options available to women in such situations.
4. Dependence on male relatives: Austen emphasizes the dependence of women on male relatives for financial security and social status. This theme is evident in both "Pride and Prejudice" and "Sense and Sensibility."

Change in Response Mode: In the "tree_summarize" response mode, the model organizes the retrieved information in a hierarchical or tree-like structure. It generates a summary that captures the main points and their relationships, often providing a more structured overview of the topic.

In [15]:
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine.query("What is Austen's Critique of Patriarchal Societies?")

display_response(response)

**`Final Response:`** Based on the information provided in the sources, Jane Austen's critique of patriarchal societies can be summarized as follows:

1. Limited Marriage Prospects: Austen highlights the economic dependence of women on male relatives and the limited marriage prospects they faced in patriarchal societies. In "Pride and Prejudice," the Bennet sisters are pressured to marry well due to their limited inheritance rights.
2. Gender Roles and Expectations: Austen critiques the rigid gender roles and expectations placed on women during the Regency era. In "Sense and Sensibility," the Dashwood sisters must navigate societal expectations and financial dependence on male relatives after their father's death.
3. Patriarchal Power Dynamics: Austen's works examine the power dynamics between men and women in patriarchal societies. In "Pride and Prejudice," Charlotte Lucas marries Mr. Collins for financial stability, illustrating the economic necessity of marriage for women of their social class.
4. Feminist Perspectives: Austen's novels provide insightful commentary on the limitations placed on women's autonomy and